In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from sanook_scraping_function import web_loading, find_title_date_views_driver, dowload_web_to_text
from tqdm import tqdm
import os 

In [15]:
urls = {
        'อาชญากรรม':'https://www.sanook.com/news/tag/อาชญากรรม/', 
        'การเมือง': 'https://www.sanook.com/news/tag/การเมือง/', 
        'ข่าวการเงิน': 'https://www.sanook.com/money/archive/', 
        'ข่าวไอที': 'https://www.sanook.com/hitech/archive/', 
        'ข่าวกีฬา': 'https://www.sanook.com/sport/archive/', 
        'สุขภาพ': 'https://www.sanook.com/health/archive/', 
        'ดูดวง': 'https://www.sanook.com/horoscope/archive/',
        'ข่าวบันเทิง': 'https://www.sanook.com/news/archive/entertain/',
        'ข่าวรถยนต์': 'https://www.sanook.com/auto/archive/',
        'ข่าวเกมส์': 'https://www.sanook.com/game/archive/',
        }

In [ ]:
search_title = "//div[@class='jsx-3504035561 archive-post-col col-12 col-md-6']/article/div[2]/div/h3/span/a"
search_date = "//div[@class='jsx-3504035561 archive-post-col col-12 col-md-6']/article/div[2]/footer/time/span"
search_view = "//div[@class='jsx-3504035561 archive-post-col col-12 col-md-6']/article/div[2]/footer/small/span[2]"

In [ ]:
# def dowload_web_to_text(url, file_path):
#     '''
#     url : link to news page (final link)
#     '''
#     page = requests.get(url)
#     soup = BeautifulSoup(page.text, 'html.parser')
#     try:
#         text_document = soup.find('div', id='EntryReader_0').text
#     except Exception as e:
#         text_document= ''
#         print(f'error to scrape this page text {url} because', e)
        
#     # return text_document
#     with open(file_path, 'w', encoding="utf-8") as f:
#         f.write(text_document)

In [ ]:
text_folder = './text_files/'
os.makedirs(text_folder, exist_ok=True)

In [ ]:
for name, url in zip(urls.keys(), urls.values()): # zip([''], ['https://www.sanook.com/horoscope/archive/']): # 
    file_name = 1
    results = {
        'Title':[],
        'Link': [],
        'Date': [],
        'Views': [],
        'File_name': [],
        'Label': []
    }
    
    print(name)
    # load web page
    driver = web_loading(url=url, scroll_pause_time=0.2, load_round=500, headless=False)

    # extract information (driver form)
    titles_dri, dates_dri, views_dri = find_title_date_views_driver(driver=driver, search_title=search_title, 
                                                                search_date=search_date, search_view=search_view)

    # driver to text
    print(f'number of records: {len(titles_dri)}')
    for title_dri, date_dri, view_dri in tqdm(zip(titles_dri, dates_dri, views_dri)):
        link = title_dri.get_attribute('href')
        results['Views'].append(view_dri.text)
        results['Link'].append(link)
        
        # ret = retrive_page_info(url=link)
        # text = retrieve_page_text(link)
        # results['Document'].append(text)
        dowload_web_to_text(link, file_path=os.path.join(text_folder, name+'_'+str(file_name)+'.txt'))
        results['File_name'].append(name+'_'+str(file_name)+'.txt')
        
        results['Title'].append(title_dri.text)
        results['Date'].append(date_dri.text)
        # title = ret.retrieve_title()
        # results['Title'].append(title)
        
        # datetime = ret.retrieve_datetime()
        # results['Datetime'].append(datetime)
        
        results['Label'].append(name)
        
        file_name += 1
        
    df = pd.DataFrame(results)
    df.to_csv(f'sanook_news_{name}.csv', index=False, encoding='utf-8-sig')

In [16]:
import pandas as pd
import glob

In [ ]:
li = []
for csv_file in glob.glob('sanook_news_*.csv'):
    df = pd.read_csv(csv_file).dropna()
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.info()

In [ ]:
df[df['Label']=='ข่าวรถยนต์']

In [ ]:
df['Label'].value_counts()

In [ ]:
df.to_csv('sanook_news_all.csv', index=False, encoding='utf-8-sig')